In [ ]:
%matplotlib notebook
from functools import cache
from math import cos, sin, pi
import odeanimate.plots.axes
from matplotlib.figure import Figure
import ipywidgets as widgets
from IPython.display import display, Math, Latex
from odeanimate.curve import Curve1D, Curve2D
from odeanimate.domains import Interval
from odeanimate.vector import Vector2D

In [ ]:
interval = Interval(0, 10)


@Curve1D
def g(x):
    """$g(x) = (\\frac{1}{300})(x-1)(x-3)(x-6)(x-9)$"""
    return (x - 2) * (x - 3) * (x - 6) * (x - 9) / 300


g_prime = g.derivative()

_t = widgets.FloatSlider(
    description="$t_0$",
    value=interval.limits[0],
    min=interval.limits[0],
    max=interval.limits[1],
    step=0.05,
)

trayectory = g.map(interval)


def interactive(t_0):
    fig = Figure(figsize=(4, 4))
    ax = fig.add_subplot(projection="odeanimate")
    ax.set_xlim(interval)
    ax.set_ylim(min(trayectory.y), max(trayectory.y))
    ax.add(trayectory)
    point = Vector2D(t_0, g(t_0))
    ax.add(point, point=True)

    m = g_prime(t_0)
    b = g(t_0)

    @Curve1D
    def tangent(t):
        return m * (t - t_0) + b

    ax.add(tangent, interval=interval, delta=1)
    display(fig)


out = widgets.interactive_output(
    interactive,
    {
        "t_0": _t,
    },
)

widgets.HBox([widgets.VBox([widgets.HTMLMath(g.__doc__), _t]), out])

In [ ]:
interval = Interval(-2, 2)


@Curve1D
def g(x):
    """$g(x) = -1 - x^2$"""
    return 3 - (1 + x**2)


g_prime = g.derivative()

n = widgets.IntSlider(
    description="$n$",
    value=1,
    min=1,
    max=150,
    step=1,
)
_type = widgets.RadioButtons(
    options=["left", "mid-point", "right"],
    value="left",
    description="Which type of Riemann Sum:",
)


def interactive(n, sum_type):
    fig = Figure(figsize=(4, 4))
    ax = fig.add_subplot()
    ax.set_xlim(*interval.limits)
    trayectory = g.map(interval)
    ax.set_ylim(min(trayectory.y), max(trayectory.y))
    ax.plot(trayectory.x, trayectory.y)
    ax.spines["top"].set_color("none")
    ax.spines["bottom"].set_position("zero")
    ax.spines["left"].set_position("zero")
    ax.spines["left"].set_color("none")
    ax.spines["right"].set_color("none")

    a, b = interval.limits
    width = (b - a) / n
    partition = [a + i * width for i in range(n + 1)]
    domain = []
    codomain = []
    for _p, _n in zip(partition, partition[1:]):
        _domain = [_p, _p, _n, _n]
        if sum_type == "mid-point":
            h = g((_p + _n) / 2)
        elif sum_type == "right":
            h = g(_n)
        else:  # sum_type == 'left'
            h = g(_p)
        _codomain = [0, h, h, 0]
        domain.extend(_domain)
        codomain.extend(_codomain)
    ax.plot(domain, codomain)
    display(fig)


out = widgets.interactive_output(interactive, {"n": n, "sum_type": _type})

widgets.HBox([widgets.VBox([widgets.HTMLMath(g.__doc__), n, _type]), out])

In [ ]:
def Parametric2DCurveFigure(function, interval, delta):
    trayectory = function.map(interval, delta)

    x_min, x_max = [fun(trayectory.x) for fun in [min, max]]
    x_delta = (x_max - x_min) / 8
    x_interval = Interval(x_min - x_delta, x_max + x_delta)
    y_min, y_max = [fun(trayectory.y) for fun in [min, max]]
    y_delta = (y_max - y_min) / 8
    y_interval = Interval(y_min - y_delta, y_max + y_delta)

    fig = Figure(figsize=(8, 4))
    gs = fig.add_gridspec(2, 4, hspace=0.2, wspace=0.2)

    ax = fig.add_subplot(gs[:, 0:2], projection="odeanimate")
    ax.set_limits(x_interval, y_interval)
    ax.add(trayectory)

    ax_x = fig.add_subplot(gs[0, 2:], projection="odeanimate")
    ax_x.set_limits(interval, x_interval)
    ax_x.add(trayectory, keys=["t", "x"])

    ax_y = fig.add_subplot(gs[1, 2:], projection="odeanimate")
    ax_y.set_limits(interval, y_interval)
    ax_y.add(trayectory, keys=["t", "y"])

    return fig, (ax, ax_x, ax_y)

In [ ]:
@Curve2D
def function(t):
    return cos(2 * pi * t), sin(2 * pi * t)


interval = Interval(0, 1)
delta = 0.01
fig, axes = Parametric2DCurveFigure(function, interval, delta)
fig

In [ ]:
widget_kwargs = dict(
    value=1,
    min=-10,
    max=10,
    step=0.05,
)
_a = widgets.FloatSlider(
    description="a",
    **widget_kwargs,
)

@cache
def interactive_function(a):
    @Curve2D
    def CissoidOfDiocles(t):
        denom = 1 + t**2
        factor = 2 * a * (t**2) / denom
        return factor, factor * t

    function = CissoidOfDiocles
    interval = Interval(-1, 1)
    delta = 0.1
    fig, axes = Parametric2DCurveFigure(function, interval, delta)
    display(fig)


out = widgets.interactive_output(interactive_function, {"a": _a})

widgets.HBox(
    [
        widgets.VBox(
            [
                _a,
            ]
        ),
        out,
    ]
)

In [ ]:
@Curve2D
def function(t):
    return t**3 - 4 * t, t**2 - 4


interval = Interval(-3, 3)
delta = 0.01
fig, axes = Parametric2DCurveFigure(function, interval, delta)
fig